In [3]:
import tensorflow as tf
from tensorflow.keras import layers,losses,metrics,optimizers


#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)

    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))

    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)
Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

#构建输入数据管道
ds = tf.data.Dataset.from_tensor_slices((X,Y)) \
     .shuffle(buffer_size = 1000).batch(100) \
     .prefetch(tf.data.experimental.AUTOTUNE)  

#定义优化器
optimizer = optimizers.SGD(learning_rate=0.001)
linear = layers.Dense(units = 1)
linear.build(input_shape = (2,)) 

@tf.function
def train(epoches):
    for epoch in tf.range(1,epoches+1):
        L = tf.constant(0.0) #使用L记录loss值
        for X_batch,Y_batch in ds:
            with tf.GradientTape() as tape:
                Y_hat = linear(X_batch)
                loss = losses.mean_squared_error(tf.reshape(Y_hat,[-1]),tf.reshape(Y_batch,[-1]))
            grads = tape.gradient(loss,linear.variables)
            print(linear.variables)
            optimizer.apply_gradients(zip(grads,linear.variables))
            L = loss

        if(epoch%100==0):
            printbar()
            tf.print("epoch =",epoch,"loss =",L)
            tf.print("w =",linear.kernel)
            tf.print("b =",linear.bias)
            tf.print("")

train(500)

[<tf.Variable 'kernel:0' shape=(2, 1) dtype=float32>, <tf.Variable 'bias:0' shape=(1,) dtype=float32>]
[<tf.Variable 'kernel:0' shape=(2, 1) dtype=float32>, <tf.Variable 'bias:0' shape=(1,) dtype=float32>]
================================================================================21:26:58
epoch = 100 loss = 4.43261862
w = [[1.99074423]
 [-1.01939857]]
b = [2.35030723]

================================================================================21:26:59
epoch = 200 loss = 4.67358732
w = [[1.9904654]
 [-1.02432048]]
b = [2.83390069]

================================================================================21:27:00
epoch = 300 loss = 4.10570049
w = [[1.99302542]
 [-1.02482557]]
b = [2.9319396]

================================================================================21:27:01
epoch = 400 loss = 3.73970652
w = [[1.9929719]
 [-1.02420795]]
b = [2.951931]

================================================================================21:27:03
epoch = 500 loss = 2.98329